In [1]:
import sqlite3
import numpy as np
from scipy.spatial import distance #https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html


## Rubrik
Syftet med den här är att hitta ett medelvärde för varje huvudgrupp. Sen kan man skicka in varje livsmedel ett i taget och se om det passar i den huvudgruppen eller i någon annan. Då kan man göra listor över vilka som passar och vilka som byter, och vart de byter.

In [2]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select * from livs limit 4000')
for row in rows:
        result.append(row)

db_contents = np.array(result)
print (db_contents)

conn.close()

[['Talg nöt' '1' 656.3 ..., 'Övrigt fett (ister, talg, kokosfett)' 86 None]
 ['Späck gris' '2' 763.0 ..., 'Övrigt fett (ister, talg, kokosfett)' 86
  None]
 ['Ister gris' '3' 884.3 ..., 'Övrigt fett (ister, talg, kokosfett)' 86
  None]
 ..., 
 ['Havredryck oberikad' '5964' 40.6 ...,
  'Vegetabiliska produkter och mjölkersättning' 46 None]
 ['Kryddblandning taco' '5973' 372.6 ..., 'Kryddor' 112 None]
 ['Tortilla wrap' '5974' 313.7 ..., 'Mjukt bröd' 12 'train']]


In [3]:
columns = np.array([ 2,  4,  5,  6])
# Possible columns: [ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57]


In [4]:
dataset = db_contents[:,columns].astype(float)

dataset är variabeln som håller sånt man kan räkna på. Alltså inte oönskade kolumner som namn eller huvudgrupp.

Man kan räkna medelvärdet över alla kategorier.

In [5]:
means = np.mean(dataset, axis=0)
means

array([ 200.13850575,   17.40143199,   10.22464559,    8.39439176])

I nästa cell tas listan över huvudgrupperna fram. 

In [6]:
label_types = np.array(list(set(db_contents.T[60]))).reshape(-1, 1) 
#https://stackoverflow.com/questions/36009907/numpy-reshape-1d-to-2d-array-with-1-column
#Man kan lägga till sorted(list(set...)) om man vill ha listan sorterad, men det blir ändå inte svensk alfabetism

In [7]:
#Lägg till en kolumn till label_types för varje kolumn, för där ska medelvärdena ligga
label_types=np.hstack((label_types,np.empty([len(label_types),len(columns)])))

In [8]:
label_types[:,0]

array(['', 'Grönsaksblandningar med rotfrukter och eller baljväxter',
       'Ättika vinäger', 'Matgryn', 'Chips popcorn o dyl',
       'Frukt o bär konserverade', 'Te', 'Blodprodukter o rätter',
       'Lightdrycker u energi', 'Inälvor organ produkter o rätter',
       'Kött färskt fryst tillagat', 'Fruktjuice mm',
       'Senap ketchup HP-sås soja *smaksättare*', 'Sås dressing majonnäs',
       'Likör', 'Godis ej choklad', 'Kryddor', 'Grädde creme fraiche',
       'Starksprit', 'Jäst bakpulver',
       'Övrigt animaliskt *kött*, grodlår, sniglar, säl - färskt, fryst, tillagat',
       'Ris risnudlar', 'Naturell fil yoghurt',
       'Hamburgare med  bröd (kött, fisk, fågel, vegetarisk)', 'Grönsaker',
       'Pizza paj pirog färdig smörgås', 'Riskakor',
       'Fisk färsk fryst kokt', 'Gröt', 'Sötningsmedel',
       'Kost- o näringspreparat', 'Smörgåskex',
       'Sojaprotein veteprotein Quorn produkter och rätter',
       'Inälvor och organ', 'Majonnässallad röror', 'Gelatin agar agar

In [9]:
a = np.where(db_contents[:,60]==label_types[2][0])

In [10]:
db_contents[a]

array([['Ättiksprit ättiksyra 12 %', '1965', 48.8, 204.0, 12.0, 0.0, 0.0,
        0.0, 88.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '', 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, '', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '0', 0.0,
        31.0, 'Ättika vinäger', 100, None],
       ['Vinäger ättiksyra 7 %', '1966', 30.1, 125.8, 7.0, 0.0, 0.4, 0.0,
        92.6, 0.0, 0.0, 0.6, 0.0, 0.0, 0.0, '', 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, '', 0.0, 0.0, 0.0, 0.0, 0.07, 0.0, 0.0,
        0.0, 32.0, '', 0.5, 15.0, 89.0, 22.0, 20.0, 0.05, 0.0, '0.1', 0.0,
        5.0, 'Ättika vinäger', 100, None],
       ['Äppelcidervinäger ättiksyra 7 %', '1967', 30.1, 125.8, 7.0, 0.0,
        0.4, 0.0, 92.6, 0.0, 0.0, 0.6, 0.0, 0.0, 0.0, '', 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0

In [11]:
label_types[:,0]

array(['', 'Grönsaksblandningar med rotfrukter och eller baljväxter',
       'Ättika vinäger', 'Matgryn', 'Chips popcorn o dyl',
       'Frukt o bär konserverade', 'Te', 'Blodprodukter o rätter',
       'Lightdrycker u energi', 'Inälvor organ produkter o rätter',
       'Kött färskt fryst tillagat', 'Fruktjuice mm',
       'Senap ketchup HP-sås soja *smaksättare*', 'Sås dressing majonnäs',
       'Likör', 'Godis ej choklad', 'Kryddor', 'Grädde creme fraiche',
       'Starksprit', 'Jäst bakpulver',
       'Övrigt animaliskt *kött*, grodlår, sniglar, säl - färskt, fryst, tillagat',
       'Ris risnudlar', 'Naturell fil yoghurt',
       'Hamburgare med  bröd (kött, fisk, fågel, vegetarisk)', 'Grönsaker',
       'Pizza paj pirog färdig smörgås', 'Riskakor',
       'Fisk färsk fryst kokt', 'Gröt', 'Sötningsmedel',
       'Kost- o näringspreparat', 'Smörgåskex',
       'Sojaprotein veteprotein Quorn produkter och rätter',
       'Inälvor och organ', 'Majonnässallad röror', 'Gelatin agar agar

In [13]:
for row in label_types:
    print(row[0])
    a = np.where(db_contents[:,60]==row[0]) # a blir index för alla livsmedel som tillhör den aktuella huvudgruppen
    #print(a)
    #print(dataset[a,0])
    x=np.mean(dataset[a], axis=0) # blir medelvärdena för alla kolumner för den aktuella huvudgruppen
    #print (x)
    row[1:]=x # Sätt medelvärdena på plats i den stora matrisen som är label_types/huvudgrupper
    #print('row',row)


Grönsaksblandningar med rotfrukter och eller baljväxter
Ättika vinäger
Matgryn
Chips popcorn o dyl
Frukt o bär konserverade
Te
Blodprodukter o rätter
Lightdrycker u energi
Inälvor organ produkter o rätter
Kött färskt fryst tillagat
Fruktjuice mm
Senap ketchup HP-sås soja *smaksättare*
Sås dressing majonnäs
Likör
Godis ej choklad
Kryddor
Grädde creme fraiche
Starksprit
Jäst bakpulver
Övrigt animaliskt *kött*, grodlår, sniglar, säl - färskt, fryst, tillagat
Ris risnudlar
Naturell fil yoghurt
Hamburgare med  bröd (kött, fisk, fågel, vegetarisk)
Grönsaker
Pizza paj pirog färdig smörgås
Riskakor
Fisk färsk fryst kokt
Gröt
Sötningsmedel
Kost- o näringspreparat
Smörgåskex
Sojaprotein veteprotein Quorn produkter och rätter
Inälvor och organ
Majonnässallad röror
Gelatin agar agar
Söta soppor kräm o efterrättssås
Flingor - frukostflingor
Bullar kakor tårtor mm
Grönsaks- rotfrukts- baljväxträtter och produkter
Övrigt
Hård matfettsblandning
Bär färska frysta
Korv
Frukt färsk fryst
Vin
Smältost
Fr

In [13]:
label_types.T

array([['', 'Grönsaker', 'Baljväxter (bönor, linser och ärter)',
        'Pastarätter', 'Te', 'Likör', 'Mjölk', 'Tacoskal', 'Smör',
        'Cider alkoläsk drink', 'Mesvaror', 'Potatisprodukter o rätter',
        'Sockerfritt godis', 'Färskost o kvarg', 'Kryddor',
        'Sås dressing majonnäs', 'Potatis', 'Inälvor och organ', 'Välling',
        'Kött processat', 'Skaldjur bläckfisk färsk fryst kokt',
        'Pannkakor, våfflor, crêpes', 'Grönsaksjuice rotfruktsjuice',
        'Övrigt', 'Inälvor organ produkter o rätter',
        'Fisk stekt ej panerad', 'Rom, kaviar', 'Majonnässallad röror',
        'Frukt o bär konserverade', 'Ättika vinäger',
        'Bullar kakor tårtor mm', 'Olja', 'Gröt',
        'Senap ketchup HP-sås soja *smaksättare*', 'Hård ost m.m.',
        'Naturell fil yoghurt', 'Pizza paj pirog färdig smörgås', 'Ägg',
        'Sportdrycker energidrycker', 'Riskakor', 'Buljong',
        'Övrigt animaliskt *kött*, grodlår, sniglar, säl - färskt, fryst, tillagat',
       

Nu är det dags att ta varje livsmedel och se vilken huvudgrupp det hamnar i:

In [14]:
ett_livsmedel = db_contents[169]
ett_livsmedel

array(['Vitt bröd tortilla fibrer 4,6%', '203', 306.5, 1282.6, 50.6, 6.8,
       7.88, 4.6, 27.6, 0.0, 2.5, 0.0, 1.72, 0.49, 0.0, '', 2.54, 0.0,
       0.01, 0.05, 2.12, 0.32, 0.02, 2.36, 0.01, 2.33, 0.96, 0.92, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.2, 2.0, 0.63, 1.2, 1.2, 0.12, 0.03,
       0.0, 0.9, 1.7, 0.08, 0.0, 17.1, 211.0, 6.0, 0.66, 48.0, 137.0, 18.0,
       685.0, 1.71, 5.0, '0.56', 0.0, 117.0, 'Mjukt bröd', 12, 'test'], dtype=object)

In [15]:
np.array([ett_livsmedel[columns]]).shape

(1, 4)

In [16]:
label_types[:,1:].shape

(119, 4)

In [17]:
dist = distance.cdist([ett_livsmedel[columns]], label_types[:,1:], 'euclidean') #https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html

In [18]:
dist

array([[  98.33121783,  268.16280448,  179.68144336,  185.3594941 ,
         231.00366236,   41.63035698,  188.80630886,  122.85184614,
         431.05320955,  251.52229373,   22.60537356,  155.26860018,
         107.34670186,  126.66871407,   69.77703777,   93.02132088,
         217.7217627 ,  145.07377356,   94.51965578,  123.79433404,
         228.42957127,  122.06023083,  287.21085638,  310.82301137,
         146.86566139,  123.87815513,  111.7621212 ,   95.06914745,
         231.90498305,  273.59209613,   96.3238918 ,  579.52312906,
         227.97839066,  186.14821539,   55.8972334 ,  250.19409239,
          76.99408518,  134.91912425,  275.79647732,  123.13547671,
         158.64927231,  242.77364025,   61.15952613,  140.92870519,
         145.57544699,  257.60001594,  230.63288471,   71.36170783,
         128.47171499,  178.61361997,  283.93774897,   62.20641489,
          79.26541089,  499.91240285,  217.8978446 ,  111.72774743,
         101.8258533 ,  240.6811883 ,   94.93297

In [19]:
order = np.argsort(dist)

In [20]:
order

array([[ 10,  87,  80,  67,   5, 105, 111,  34,  64,  99,  42,  51,  14,
         73,  47,  95,  36,  52, 118,  92,  78, 108, 112,  97,  98,  15,
         81, 117,  18,  58,  27,  30,   0,  56,  59, 100,  12,  55,  26,
         76,  94,  65,  21,   7,  39, 106,  19,  25,  68,  13,  48, 110,
        107,  37,  88,  43,  17, 102,  44,  24,  11,  90,  96,  40, 103,
         72,  91,  74,  49,   2,  93,   3,  33,   6, 114, 101,  77,  16,
         54,  70,  75,  32,  20,  46,   4,  28, 109,  86,  57,  41, 115,
        104,  35,  61,  83,  85,   9,  69,  62,  60,  45, 113,  79,  82,
          1,  29,  38,  84, 116,  50,  89,  71,  22,  63,  23,  66,   8,
         53,  31]])

In [21]:
label_types[order,0]

array([['Mesvaror', 'Frukt o bär torkade', 'Mjukt bröd',
        'Mjöl stärkelse kli', 'Likör', 'Gelatin agar agar', 'Algprodukter',
        'Hård ost m.m.', 'Hårt bröd', 'Dessertost',
        'Hamburgare med  bröd (kött, fisk, fågel, vegetarisk)',
        'Ris risnudlar', 'Kryddor', 'Tuggummi', 'Socker sirap honung',
        'Korv', 'Pizza paj pirog färdig smörgås', 'Godis ej choklad',
        'Pasta', 'Ost med vegetabilskt fett', 'Flingor - frukostflingor',
        'Fisk rökt', 'Deg och gräddade skal och bottnar', 'Kakaoprodukter',
        'Blodprodukter o rätter', 'Sås dressing majonnäs', 'Starksprit',
        'Jäst bakpulver', 'Välling', 'Fågelprodukter o rätter',
        'Majonnässallad röror', 'Bullar kakor tårtor mm', '', 'Glass',
        'Frukt o nötblandningar bars', 'Osträtter', 'Sockerfritt godis',
        'Grädde creme fraiche', 'Rom, kaviar', 'Sötningsmedel',
        'Efterrätter',
        'Sojaprotein veteprotein Quorn produkter och rätter',
        'Pannkakor, våfflor, c

In [22]:
np.where(label_types[order,0] == ett_livsmedel[60])[1][0]+1 # Vilken plats kom ursprungskategorin på?

3

Dags att göra en funktion av det:

In [23]:
def predict_category (index):
    ett_livsmedel = db_contents[index]
    dist = distance.cdist([ett_livsmedel[columns]], label_types[:,1:], 'euclidean') #https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html
    order = np.argsort(dist)
    closest = label_types[order,0]
    orig_category_place = np.where(label_types[order,0] == ett_livsmedel[60])[1][0]+1
    return ett_livsmedel[0], index, orig_category_place, closest[0][0], ett_livsmedel[60]

In [24]:
a = np.array([])
for index, foodstuff in enumerate(db_contents):
    if (predict_category (index))[2] > 100:
        #print (predict_category (index))
        a = np.append(a, list(predict_category (index)), axis=0)
        #print ()
a

array(['Kastanjer', '1322', '114', 'Glass', 'Nötter frön',
       'Kastanjer rostade', '1336', '114', 'Pasta', 'Nötter frön',
       'Gräddtårta hembakad jordgubb banan', '1412', '102', 'Glass',
       'Bullar kakor tårtor mm', 'Petits-choux u fyllning', '1421', '103',
       'Efterrätter', 'Bullar kakor tårtor mm', 'Katrinplommonkaka',
       '1488', '104', 'Matgryn', 'Bullar kakor tårtor mm',
       'Lättmargarin fett 28% berikad typ Mini Lätta', '1858', '103',
       'Korv', 'Hård matfettsblandning'],
      dtype='<U44')

In [25]:
a = a.reshape(int(len(a)/5),5)

In [26]:
a.shape

(6, 5)

In [27]:
for row in a:
    print(row)

['Kastanjer' '1322' '114' 'Glass' 'Nötter frön']
['Kastanjer rostade' '1336' '114' 'Pasta' 'Nötter frön']
['Gräddtårta hembakad jordgubb banan' '1412' '102' 'Glass'
 'Bullar kakor tårtor mm']
['Petits-choux u fyllning' '1421' '103' 'Efterrätter'
 'Bullar kakor tårtor mm']
['Katrinplommonkaka' '1488' '104' 'Matgryn' 'Bullar kakor tårtor mm']
['Lättmargarin fett 28% berikad typ Mini Lätta' '1858' '103' 'Korv'
 'Hård matfettsblandning']
